# 🫀 Heart Segmentation Advanced - Data Analysis & Preprocessing

<a href="https://colab.research.google.com/github/leonardobora/pratica-aprendizado-de-maquina/blob/main/Heart_Segmentation_Advanced/01_Data_Analysis_and_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📋 Objetivos deste Notebook

Este notebook realiza análise exploratória detalhada e pré-processamento dos dados cardíacos:

- 📊 **Análise exploratória** do dataset Task02_Heart
- 🔍 **Investigação das estruturas** de dados NIfTI
- 📐 **Análise de dimensões** e propriedades espaciais
- 🏷️ **Análise de distribuição** de classes
- 🔧 **Pipeline de pré-processamento** otimizado
- 💾 **Preparação de dados** para treinamento

---

**⚠️ PRÉ-REQUISITO**: Execute primeiro `00_Setup_and_Configuration.ipynb`

In [12]:
# =============================================================================
# 📚 CARREGAR CONFIGURAÇÕES DO SETUP
# =============================================================================

# Executar setup básico se não foi executado
try:
    # Tentar carregar configurações salvas
    import json
    import os
    
    # Assumir que estamos no diretório do projeto ou ajustar path
    if 'Heart_Segmentation_Advanced' not in os.getcwd():
        # Se não estivermos no diretório correto, ajustar
        os.chdir('/content/drive/MyDrive/Heart_Segmentation_Advanced')
    
    with open('project_config.json', 'r') as f:
        project_config = json.load(f)
    
    print("✅ Configurações carregadas do setup anterior")
    
except FileNotFoundError:
    print("⚠️ Configurações não encontradas. Execute primeiro 00_Setup_and_Configuration.ipynb")
    print("🔄 Executando setup básico...")
    
    # Setup mínimo necessário
    exec(open('00_Setup_and_Configuration.ipynb').read())

✅ Configurações carregadas do setup anterior


In [2]:
# =============================================================================
# 📚 IMPORTS PARA ANÁLISE DE DADOS
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import SimpleITK as sitk
import os
import glob
from pathlib import Path
import json
from collections import defaultdict
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Estatísticas
from scipy import stats
from scipy.ndimage import label, center_of_mass

# Visualização avançada
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("📚 Imports realizados com sucesso!")

📚 Imports realizados com sucesso!


In [13]:
# =============================================================================
# 🔍 DESCOBERTA E INVENTÁRIO DO DATASET
# =============================================================================

def discover_dataset(dataset_path):
    """Descobre e analisa a estrutura do dataset"""
    
    print("🔍 DESCOBRINDO ESTRUTURA DO DATASET")
    print("=" * 60)
    
    if not os.path.exists(dataset_path):
        print(f"❌ Dataset não encontrado em: {dataset_path}")
        print("   Certifique-se de que o dataset Task02_Heart está no local correto")
        return None
    
    print(f"📁 Dataset encontrado: {dataset_path}")
    
    # Explorar estrutura de diretórios
    dataset_info = {
        'base_path': dataset_path,
        'subdirs': [],
        'files': []
    }
    
    for root, dirs, files in os.walk(dataset_path):
        level = root.replace(dataset_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        
        sub_indent = ' ' * 2 * (level + 1)
        for file in files:
            if file.endswith('.nii.gz'):
                print(f"{sub_indent}📄 {file}")
                dataset_info['files'].append(os.path.join(root, file))
    
    # Verificar diretórios essenciais
    essential_dirs = ['imagesTr', 'labelsTr']
    for dir_name in essential_dirs:
        dir_path = os.path.join(dataset_path, dir_name)
        if os.path.exists(dir_path):
            files_count = len([f for f in os.listdir(dir_path) if f.endswith('.nii.gz')])
            print(f"✅ {dir_name}: {files_count} arquivos")
            dataset_info['subdirs'].append({
                'name': dir_name,
                'path': dir_path,
                'file_count': files_count
            })
        else:
            print(f"❌ {dir_name}: não encontrado")
    
    return dataset_info

# Executar descoberta
DATASET_PATH = project_config['paths']['dataset']
dataset_info = discover_dataset(DATASET_PATH)

🔍 DESCOBRINDO ESTRUTURA DO DATASET
📁 Dataset encontrado: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\Task02_Heart
📁 Task02_Heart/
  📁 imagesTr/
    📄 ._la_029.nii.gz
    📄 la_003.nii.gz
    📄 la_004.nii.gz
    📄 la_005.nii.gz
    📄 la_007.nii.gz
    📄 la_009.nii.gz
    📄 la_010.nii.gz
    📄 la_011.nii.gz
    📄 la_014.nii.gz
    📄 la_016.nii.gz
    📄 la_017.nii.gz
    📄 la_018.nii.gz
    📄 la_019.nii.gz
    📄 la_020.nii.gz
    📄 la_021.nii.gz
    📄 la_022.nii.gz
    📄 la_023.nii.gz
    📄 la_024.nii.gz
    📄 la_026.nii.gz
    📄 la_029.nii.gz
    📄 la_030.nii.gz
  📁 imagesTs/
    📄 la_001.nii.gz
    📄 la_002.nii.gz
    📄 la_006.nii.gz
    📄 la_008.nii.gz
    📄 la_012.nii.gz
    📄 la_013.nii.gz
    📄 la_015.nii.gz
    📄 la_025.nii.gz
    📄 la_027.nii.gz
    📄 la_028.nii.gz
  📁 labelsTr/
    📄 ._la_014.nii.gz
    📄 ._la_029.nii.gz
    📄 la_003.nii.gz
    📄 la_004.nii.gz
    📄 la_005.nii.gz
    📄 la_007.nii.gz


In [14]:
# =============================================================================
# 📊 ANÁLISE DETALHADA DOS ARQUIVOS
# =============================================================================

def analyze_nifti_files(dataset_path):
    """Analisa arquivos NIfTI em detalhes"""
    
    images_dir = os.path.join(dataset_path, 'imagesTr')
    labels_dir = os.path.join(dataset_path, 'labelsTr')
    
    if not (os.path.exists(images_dir) and os.path.exists(labels_dir)):
        print("❌ Diretórios de imagens e/ou labels não encontrados")
        return None
    
    # Coletar arquivos
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.nii.gz')])
    label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.nii.gz')])
    
    print(f"📊 Encontrados {len(image_files)} arquivos de imagem")
    print(f"📊 Encontrados {len(label_files)} arquivos de label")
    
    # Analisar correspondência
    image_ids = [f.replace('_0000.nii.gz', '') for f in image_files if '_0000' in f]
    label_ids = [f.replace('.nii.gz', '') for f in label_files]
    
    matched_pairs = set(image_ids) & set(label_ids)
    print(f"✅ {len(matched_pairs)} pares imagem-label correspondentes")
    
    if len(matched_pairs) != len(image_ids):
        missing = set(image_ids) - matched_pairs
        print(f"⚠️ Imagens sem labels correspondentes: {missing}")
    
    # Análise detalhada de amostras
    analysis_results = {
        'files_info': [],
        'dimensions': [],
        'spacings': [],
        'orientations': [],
        'value_ranges': [],
        'class_distributions': []
    }
    
    print("\n🔬 Analisando arquivos em detalhes...")
    
    # Analisar primeiro algumas amostras para obter estatísticas
    sample_size = min(5, len(matched_pairs))
    sample_ids = list(matched_pairs)[:sample_size]
    
    for i, file_id in enumerate(tqdm(sample_ids, desc="Analisando amostras")):
        
        # Paths dos arquivos
        img_path = os.path.join(images_dir, f"{file_id}_0000.nii.gz")
        label_path = os.path.join(labels_dir, f"{file_id}.nii.gz")
        
        try:
            # Carregar imagem
            img_nii = nib.load(img_path)
            img_data = img_nii.get_fdata()
            
            # Carregar label
            label_nii = nib.load(label_path)
            label_data = label_nii.get_fdata()
            
            # Informações básicas
            file_info = {
                'id': file_id,
                'img_shape': img_data.shape,
                'label_shape': label_data.shape,
                'img_dtype': str(img_data.dtype),
                'label_dtype': str(label_data.dtype)
            }
            
            # Dimensões e espaçamento
            header = img_nii.header
            pixdim = header['pixdim'][1:4]  # Espaçamento dos pixels
            
            file_info.update({
                'pixel_spacing': pixdim.tolist(),
                'orientation': str(nib.aff2axcodes(img_nii.affine)),
                'img_min': float(np.min(img_data)),
                'img_max': float(np.max(img_data)),
                'img_mean': float(np.mean(img_data)),
                'img_std': float(np.std(img_data))
            })
            
            # Distribuição de classes
            unique_labels, counts = np.unique(label_data, return_counts=True)
            class_dist = dict(zip(unique_labels.astype(int), counts))
            file_info['class_distribution'] = class_dist
            
            analysis_results['files_info'].append(file_info)
            analysis_results['dimensions'].append(img_data.shape)
            analysis_results['spacings'].append(pixdim.tolist())
            analysis_results['value_ranges'].append((np.min(img_data), np.max(img_data)))
            analysis_results['class_distributions'].append(class_dist)
            
        except Exception as e:
            print(f"❌ Erro ao analisar {file_id}: {e}")
            continue
    
    return analysis_results, matched_pairs

# Executar análise
if dataset_info:
    analysis_results, matched_pairs = analyze_nifti_files(DATASET_PATH)
else:
    print("⚠️ Pulando análise - dataset não encontrado")
    analysis_results, matched_pairs = None, None

📊 Encontrados 21 arquivos de imagem
📊 Encontrados 22 arquivos de label
✅ 0 pares imagem-label correspondentes

🔬 Analisando arquivos em detalhes...


Analisando amostras: 0it [00:00, ?it/s]

In [5]:
# =============================================================================
# 📈 VISUALIZAÇÃO DE ESTATÍSTICAS DO DATASET
# =============================================================================

def visualize_dataset_statistics(analysis_results):
    """Cria visualizações das estatísticas do dataset"""
    
    if not analysis_results:
        print("❌ Sem dados para visualizar")
        return
    
    files_info = analysis_results['files_info']
    
    # Criar DataFrame para análise
    df_stats = pd.DataFrame(files_info)
    
    print("📊 ESTATÍSTICAS DO DATASET")
    print("=" * 50)
    
    # Estatísticas de dimensões
    dimensions = [info['img_shape'] for info in files_info]
    dimensions_df = pd.DataFrame(dimensions, columns=['Width', 'Height', 'Slices'])
    
    print("📐 Dimensões das imagens:")
    print(dimensions_df.describe())
    
    # Estatísticas de espaçamento
    spacings = [info['pixel_spacing'] for info in files_info]
    spacings_df = pd.DataFrame(spacings, columns=['Spacing_X', 'Spacing_Y', 'Spacing_Z'])
    
    print("\n📏 Espaçamento dos pixels:")
    print(spacings_df.describe())
    
    # Estatísticas de intensidade
    print(f"\n💡 Intensidades das imagens:")
    print(f"   Min: {df_stats['img_min'].min():.2f} - {df_stats['img_min'].max():.2f}")
    print(f"   Max: {df_stats['img_max'].min():.2f} - {df_stats['img_max'].max():.2f}")
    print(f"   Mean: {df_stats['img_mean'].min():.2f} - {df_stats['img_mean'].max():.2f}")
    print(f"   Std: {df_stats['img_std'].min():.2f} - {df_stats['img_std'].max():.2f}")
    
    # Plotar visualizações
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('📊 Análise Estatística do Dataset Task02_Heart', fontsize=16, fontweight='bold')
    
    # Distribuição de dimensões
    axes[0, 0].hist([d[2] for d in dimensions], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribuição do Número de Fatias')
    axes[0, 0].set_xlabel('Número de Fatias')
    axes[0, 0].set_ylabel('Frequência')
    
    # Distribuição de espaçamento
    axes[0, 1].boxplot([spacings_df['Spacing_X'], spacings_df['Spacing_Y'], spacings_df['Spacing_Z']], 
                      labels=['X', 'Y', 'Z'])
    axes[0, 1].set_title('Espaçamento dos Pixels por Eixo')
    axes[0, 1].set_ylabel('Espaçamento (mm)')
    
    # Distribuição de intensidades médias
    axes[0, 2].hist(df_stats['img_mean'], bins=10, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0, 2].set_title('Distribuição das Intensidades Médias')
    axes[0, 2].set_xlabel('Intensidade Média')
    axes[0, 2].set_ylabel('Frequência')
    
    # Scatter plot: dimensões vs intensidade média
    scatter_data = [(d[0], d[1], d[2], m) for d, m in zip(dimensions, df_stats['img_mean'])]
    scatter_df = pd.DataFrame(scatter_data, columns=['Width', 'Height', 'Slices', 'Mean_Intensity'])
    
    scatter = axes[1, 0].scatter(scatter_df['Slices'], scatter_df['Mean_Intensity'], 
                                alpha=0.7, c=scatter_df['Width'], cmap='viridis')
    axes[1, 0].set_title('Fatias vs Intensidade Média')
    axes[1, 0].set_xlabel('Número de Fatias')
    axes[1, 0].set_ylabel('Intensidade Média')
    plt.colorbar(scatter, ax=axes[1, 0], label='Width')
    
    # Distribuição de classes agregada
    all_class_counts = defaultdict(int)
    for class_dist in analysis_results['class_distributions']:
        for class_id, count in class_dist.items():
            all_class_counts[class_id] += count
    
    classes = list(all_class_counts.keys())
    counts = list(all_class_counts.values())
    
    axes[1, 1].bar(classes, counts, color=['black', 'red', 'green'], alpha=0.7)
    axes[1, 1].set_title('Distribuição Total de Classes')
    axes[1, 1].set_xlabel('Classe')
    axes[1, 1].set_ylabel('Contagem de Pixels')
    axes[1, 1].set_yscale('log')
    
    # Proporção de classes
    total_pixels = sum(counts)
    proportions = [count/total_pixels for count in counts]
    
    axes[1, 2].pie(proportions, labels=[f'Classe {c}' for c in classes], 
                  colors=['black', 'red', 'green'], autopct='%1.1f%%')
    axes[1, 2].set_title('Proporção de Classes')
    
    plt.tight_layout()
    plt.show()
    
    return df_stats, dimensions_df, spacings_df

# Visualizar estatísticas
if analysis_results:
    df_stats, dimensions_df, spacings_df = visualize_dataset_statistics(analysis_results)

In [6]:
# =============================================================================
# 🔬 ANÁLISE DETALHADA DE UMA AMOSTRA
# =============================================================================

def analyze_sample_in_detail(dataset_path, sample_id, matched_pairs):
    """Análise detalhada de uma amostra específica"""
    
    if not matched_pairs or sample_id not in matched_pairs:
        # Pegar primeira amostra disponível
        sample_id = list(matched_pairs)[0] if matched_pairs else None
        if not sample_id:
            print("❌ Nenhuma amostra disponível para análise")
            return None
    
    print(f"🔬 ANÁLISE DETALHADA DA AMOSTRA: {sample_id}")
    print("=" * 60)
    
    # Carregar arquivos
    images_dir = os.path.join(dataset_path, 'imagesTr')
    labels_dir = os.path.join(dataset_path, 'labelsTr')
    
    img_path = os.path.join(images_dir, f"{sample_id}_0000.nii.gz")
    label_path = os.path.join(labels_dir, f"{sample_id}.nii.gz")
    
    try:
        # Carregar usando nibabel
        img_nii = nib.load(img_path)
        label_nii = nib.load(label_path)
        
        img_data = img_nii.get_fdata()
        label_data = label_nii.get_fdata()
        
        print(f"📄 Arquivo de imagem: {os.path.basename(img_path)}")
        print(f"📄 Arquivo de label: {os.path.basename(label_path)}")
        print(f"📐 Dimensões da imagem: {img_data.shape}")
        print(f"📐 Dimensões do label: {label_data.shape}")
        
        # Informações do header
        header = img_nii.header
        print(f"🔍 Orientação: {nib.aff2axcodes(img_nii.affine)}")
        print(f"📏 Espaçamento: {header['pixdim'][1:4]}")
        print(f"🎯 Tipo de dado: {img_data.dtype}")
        
        # Estatísticas da imagem
        print(f"\n💡 Estatísticas da imagem:")
        print(f"   Min: {np.min(img_data):.2f}")
        print(f"   Max: {np.max(img_data):.2f}")
        print(f"   Mean: {np.mean(img_data):.2f}")
        print(f"   Std: {np.std(img_data):.2f}")
        print(f"   Percentis: {np.percentile(img_data, [5, 25, 50, 75, 95])}")
        
        # Análise das classes
        unique_labels, counts = np.unique(label_data, return_counts=True)
        print(f"\n🏷️ Classes encontradas:")
        class_names = ['Background', 'Left Ventricle', 'Myocardium']
        for label, count in zip(unique_labels, counts):
            label_int = int(label)
            class_name = class_names[label_int] if label_int < len(class_names) else f"Classe {label_int}"
            percentage = (count / label_data.size) * 100
            print(f"   {class_name} (Classe {label_int}): {count:,} pixels ({percentage:.2f}%)")
        
        # Visualização de fatias representativas
        visualize_sample_slices(img_data, label_data, sample_id)
        
        return img_data, label_data
        
    except Exception as e:
        print(f"❌ Erro ao carregar amostra {sample_id}: {e}")
        return None, None

def visualize_sample_slices(img_data, label_data, sample_id):
    """Visualiza fatias representativas da amostra"""
    
    num_slices = img_data.shape[2]
    
    # Selecionar fatias representativas
    slice_indices = [
        num_slices // 4,      # 25%
        num_slices // 2,      # 50% (meio)
        3 * num_slices // 4   # 75%
    ]
    
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    fig.suptitle(f'🔬 Análise da Amostra {sample_id} - Fatias Representativas', 
                fontsize=16, fontweight='bold')
    
    for i, slice_idx in enumerate(slice_indices):
        
        # Imagem original
        axes[i, 0].imshow(img_data[:, :, slice_idx].T, cmap='gray', origin='lower')
        axes[i, 0].set_title(f'Imagem - Fatia {slice_idx}')
        axes[i, 0].axis('off')
        
        # Label colorizada
        label_slice = label_data[:, :, slice_idx].T
        axes[i, 1].imshow(label_slice, cmap='jet', origin='lower', vmin=0, vmax=2)
        axes[i, 1].set_title(f'Labels - Fatia {slice_idx}')
        axes[i, 1].axis('off')
        
        # Overlay
        axes[i, 2].imshow(img_data[:, :, slice_idx].T, cmap='gray', origin='lower')
        
        # Criar máscara colorida para overlay
        label_colored = np.zeros((*label_slice.shape, 3))
        label_colored[label_slice == 1] = [1, 0, 0]  # Vermelho para classe 1
        label_colored[label_slice == 2] = [0, 1, 0]  # Verde para classe 2
        
        # Aplicar transparência
        alpha = 0.3
        overlay_mask = label_slice > 0
        label_colored = label_colored * alpha
        
        axes[i, 2].imshow(label_colored, origin='lower', alpha=0.7)
        axes[i, 2].set_title(f'Overlay - Fatia {slice_idx}')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Criar visualização interativa com Plotly
    create_interactive_volume_view(img_data, label_data, sample_id)

def create_interactive_volume_view(img_data, label_data, sample_id):
    """Cria visualização interativa do volume"""
    
    # Selecionar fatia do meio para visualização
    mid_slice = img_data.shape[2] // 2
    
    img_slice = img_data[:, :, mid_slice]
    label_slice = label_data[:, :, mid_slice]
    
    # Criar subplot
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Imagem Original', 'Segmentação', 'Histograma'),
        specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}, {'type': 'histogram'}]]
    )
    
    # Imagem original
    fig.add_trace(
        go.Heatmap(
            z=img_slice.T,
            colorscale='gray',
            showscale=True,
            name='Imagem'
        ),
        row=1, col=1
    )
    
    # Segmentação
    fig.add_trace(
        go.Heatmap(
            z=label_slice.T,
            colorscale='viridis',
            showscale=True,
            name='Labels'
        ),
        row=1, col=2
    )
    
    # Histograma
    fig.add_trace(
        go.Histogram(
            x=img_slice.flatten(),
            nbinsx=50,
            name='Distribuição de Intensidades'
        ),
        row=1, col=3
    )
    
    fig.update_layout(
        title=f'📊 Visualização Interativa - {sample_id} (Fatia {mid_slice})',
        height=500
    )
    
    fig.show()

# Executar análise detalhada
if analysis_results and matched_pairs:
    sample_img, sample_label = analyze_sample_in_detail(DATASET_PATH, None, matched_pairs)

In [7]:
# =============================================================================
# 🔧 PIPELINE DE PRÉ-PROCESSAMENTO
# =============================================================================

def create_preprocessing_pipeline():
    """Cria pipeline de pré-processamento otimizado"""
    
    class HeartDataPreprocessor:
        def __init__(self, target_size=(128, 128), normalize_method='percentile'):
            self.target_size = target_size
            self.normalize_method = normalize_method
            
        def load_volume(self, filepath):
            """Carrega volume 3D com orientação correta"""
            try:
                img = nib.load(filepath)
                data = img.get_fdata()
                
                # Corrigir orientação se necessário
                # Task02_Heart geralmente precisa de transpose
                data = np.transpose(data, (2, 1, 0))
                
                return data
            except Exception as e:
                print(f"❌ Erro ao carregar {filepath}: {e}")
                return None
        
        def normalize_slice(self, slice_img, method='percentile'):
            """Normaliza fatia individual"""
            
            if method == 'percentile':
                # Normalização baseada em percentis (melhor para imagens médicas)
                p2 = np.percentile(slice_img, 2)
                p98 = np.percentile(slice_img, 98)
                normalized = np.clip((slice_img - p2) / (p98 - p2 + 1e-8), 0, 1)
                
            elif method == 'z_score':
                # Z-score normalization
                mean = np.mean(slice_img)
                std = np.std(slice_img)
                normalized = (slice_img - mean) / (std + 1e-8)
                normalized = np.clip(normalized, -3, 3)  # Clip outliers
                normalized = (normalized + 3) / 6  # Normalize to [0, 1]
                
            elif method == 'min_max':
                # Min-max normalization
                min_val = np.min(slice_img)
                max_val = np.max(slice_img)
                normalized = (slice_img - min_val) / (max_val - min_val + 1e-8)
                
            else:
                raise ValueError(f"Método de normalização inválido: {method}")
            
            return normalized.astype(np.float32)
        
        def resize_slice(self, slice_img, target_size, order=1):
            """Redimensiona fatia preservando proporções"""
            from skimage.transform import resize
            
            resized = resize(
                slice_img, 
                target_size,
                order=order,  # 1 = bilinear, 0 = nearest neighbor
                mode='constant',
                preserve_range=True,
                anti_aliasing=True
            )
            
            return resized.astype(slice_img.dtype)
        
        def preprocess_volume(self, volume, is_label=False):
            """Pré-processa volume completo"""
            
            if volume is None:
                return None
                
            processed_slices = []
            
            for slice_idx in range(volume.shape[0]):
                slice_img = volume[slice_idx, :, :]
                
                if not is_label:
                    # Normalizar apenas imagens (não labels)
                    slice_img = self.normalize_slice(slice_img, self.normalize_method)
                
                # Redimensionar
                order = 0 if is_label else 1  # Nearest neighbor para labels
                slice_resized = self.resize_slice(slice_img, self.target_size, order=order)
                
                processed_slices.append(slice_resized)
            
            return np.array(processed_slices)
        
        def process_pair(self, img_path, label_path):
            """Processa par imagem-label"""
            
            # Carregar volumes
            img_volume = self.load_volume(img_path)
            label_volume = self.load_volume(label_path)
            
            if img_volume is None or label_volume is None:
                return None, None
            
            # Verificar compatibilidade de dimensões
            if img_volume.shape != label_volume.shape:
                print(f"⚠️ Dimensões incompatíveis: {img_volume.shape} vs {label_volume.shape}")
                return None, None
            
            # Pré-processar
            img_processed = self.preprocess_volume(img_volume, is_label=False)
            label_processed = self.preprocess_volume(label_volume, is_label=True)
            
            return img_processed, label_processed
    
    return HeartDataPreprocessor()

# Criar preprocessor
preprocessor = create_preprocessing_pipeline()
print("✅ Pipeline de pré-processamento criado")

✅ Pipeline de pré-processamento criado


In [8]:
# =============================================================================
# 🧪 TESTE DO PIPELINE DE PRÉ-PROCESSAMENTO
# =============================================================================

def test_preprocessing_pipeline(preprocessor, dataset_path, matched_pairs, num_samples=2):
    """Testa pipeline de pré-processamento"""
    
    if not matched_pairs:
        print("❌ Nenhuma amostra disponível para teste")
        return None
    
    print("🧪 TESTANDO PIPELINE DE PRÉ-PROCESSAMENTO")
    print("=" * 50)
    
    images_dir = os.path.join(dataset_path, 'imagesTr')
    labels_dir = os.path.join(dataset_path, 'labelsTr')
    
    test_results = []
    sample_ids = list(matched_pairs)[:num_samples]
    
    for sample_id in sample_ids:
        print(f"\n🔄 Processando amostra: {sample_id}")
        
        img_path = os.path.join(images_dir, f"{sample_id}_0000.nii.gz")
        label_path = os.path.join(labels_dir, f"{sample_id}.nii.gz")
        
        # Processar
        img_processed, label_processed = preprocessor.process_pair(img_path, label_path)
        
        if img_processed is not None and label_processed is not None:
            
            result = {
                'sample_id': sample_id,
                'original_shape': None,  # Será preenchido
                'processed_shape': img_processed.shape,
                'img_stats': {
                    'min': np.min(img_processed),
                    'max': np.max(img_processed),
                    'mean': np.mean(img_processed),
                    'std': np.std(img_processed)
                },
                'label_classes': np.unique(label_processed),
                'success': True
            }
            
            print(f"  ✅ Sucesso!")
            print(f"     Forma processada: {img_processed.shape}")
            print(f"     Range da imagem: [{result['img_stats']['min']:.3f}, {result['img_stats']['max']:.3f}]")
            print(f"     Classes no label: {result['label_classes']}")
            
            test_results.append(result)
            
            # Visualizar resultado
            visualize_preprocessing_result(img_processed, label_processed, sample_id)
            
        else:
            print(f"  ❌ Falha no processamento")
            test_results.append({
                'sample_id': sample_id,
                'success': False
            })
    
    return test_results

def visualize_preprocessing_result(img_processed, label_processed, sample_id):
    """Visualiza resultado do pré-processamento"""
    
    # Selecionar fatias para visualização
    num_slices = img_processed.shape[0]
    slice_idx = num_slices // 2
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f'🔧 Resultado do Pré-processamento - {sample_id}', fontweight='bold')
    
    # Imagem processada
    axes[0].imshow(img_processed[slice_idx], cmap='gray')
    axes[0].set_title(f'Imagem Processada\n{img_processed.shape}')
    axes[0].axis('off')
    
    # Label processado
    axes[1].imshow(label_processed[slice_idx], cmap='jet', vmin=0, vmax=2)
    axes[1].set_title(f'Label Processado\n{label_processed.shape}')
    axes[1].axis('off')
    
    # Overlay
    axes[2].imshow(img_processed[slice_idx], cmap='gray')
    
    # Criar overlay colorido
    label_slice = label_processed[slice_idx]
    overlay = np.zeros((*label_slice.shape, 3))
    overlay[label_slice == 1] = [1, 0, 0]  # Vermelho
    overlay[label_slice == 2] = [0, 1, 0]  # Verde
    
    axes[2].imshow(overlay, alpha=0.4)
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

# Testar pipeline
if matched_pairs:
    test_results = test_preprocessing_pipeline(preprocessor, DATASET_PATH, matched_pairs)

In [9]:
# =============================================================================
# 💾 PREPARAÇÃO DE DADOS PARA TREINAMENTO
# =============================================================================

def prepare_training_data(preprocessor, dataset_path, matched_pairs, 
                         validation_split=0.2, test_split=0.1, 
                         save_processed=True):
    """Prepara dados completos para treinamento"""
    
    if not matched_pairs:
        print("❌ Nenhuma amostra disponível")
        return None
    
    print("💾 PREPARANDO DADOS PARA TREINAMENTO")
    print("=" * 50)
    
    images_dir = os.path.join(dataset_path, 'imagesTr')
    labels_dir = os.path.join(dataset_path, 'labelsTr')
    
    # Listas para armazenar dados processados
    all_images = []
    all_labels = []
    all_sample_ids = []
    
    print(f"🔄 Processando {len(matched_pairs)} amostras...")
    
    for sample_id in tqdm(matched_pairs, desc="Processando amostras"):
        
        img_path = os.path.join(images_dir, f"{sample_id}_0000.nii.gz")
        label_path = os.path.join(labels_dir, f"{sample_id}.nii.gz")
        
        # Processar par
        img_processed, label_processed = preprocessor.process_pair(img_path, label_path)
        
        if img_processed is not None and label_processed is not None:
            all_images.append(img_processed)
            all_labels.append(label_processed)
            all_sample_ids.extend([sample_id] * len(img_processed))
    
    if not all_images:
        print("❌ Nenhuma amostra foi processada com sucesso")
        return None
    
    # Concatenar todas as fatias
    X = np.concatenate(all_images, axis=0)
    y = np.concatenate(all_labels, axis=0)
    
    print(f"✅ Dados processados:")
    print(f"   📊 Total de fatias: {len(X)}")
    print(f"   📐 Forma das imagens: {X.shape}")
    print(f"   📐 Forma dos labels: {y.shape}")
    
    # Adicionar dimensão de canal para imagens
    X = X[..., np.newaxis]
    
    # Converter labels para one-hot encoding
    y_categorical = utils.to_categorical(y, num_classes=3)
    
    print(f"   📐 Forma final X: {X.shape}")
    print(f"   📐 Forma final y: {y_categorical.shape}")
    
    # Dividir dados em treino/validação/teste
    # Primeiro, separar teste
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y_categorical, test_size=test_split, random_state=42, stratify=np.argmax(y_categorical, axis=-1)
    )
    
    # Depois, dividir treino e validação
    val_size = validation_split / (1 - test_split)  # Ajustar proporção
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_size, random_state=42, stratify=np.argmax(y_temp, axis=-1)
    )
    
    print(f"\n📊 Divisão final dos dados:")
    print(f"   🏋️ Treino: {X_train.shape[0]} fatias ({X_train.shape[0]/len(X)*100:.1f}%)")
    print(f"   ✅ Validação: {X_val.shape[0]} fatias ({X_val.shape[0]/len(X)*100:.1f}%)")
    print(f"   🧪 Teste: {X_test.shape[0]} fatias ({X_test.shape[0]/len(X)*100:.1f}%)")
    
    # Análise de distribuição de classes
    for split_name, y_split in [('Treino', y_train), ('Validação', y_val), ('Teste', y_test)]:
        class_counts = np.sum(y_split, axis=0)
        total = np.sum(class_counts)
        print(f"\n🏷️ Distribuição de classes ({split_name}):")
        for i, (count, class_name) in enumerate(zip(class_counts, 
                                                   ['Background', 'Left Ventricle', 'Myocardium'])):
            print(f"   {class_name}: {int(count):,} pixels ({count/total*100:.1f}%)")
    
    # Salvar dados processados se solicitado
    if save_processed:
        save_path = os.path.join(project_config['paths']['outputs'], 'processed_data')
        os.makedirs(save_path, exist_ok=True)
        
        print(f"\n💾 Salvando dados processados em: {save_path}")
        
        np.save(os.path.join(save_path, 'X_train.npy'), X_train)
        np.save(os.path.join(save_path, 'y_train.npy'), y_train)
        np.save(os.path.join(save_path, 'X_val.npy'), X_val)
        np.save(os.path.join(save_path, 'y_val.npy'), y_val)
        np.save(os.path.join(save_path, 'X_test.npy'), X_test)
        np.save(os.path.join(save_path, 'y_test.npy'), y_test)
        
        # Salvar metadados
        metadata = {
            'total_samples': len(matched_pairs),
            'total_slices': len(X),
            'image_shape': X.shape[1:],
            'num_classes': 3,
            'class_names': ['Background', 'Left Ventricle', 'Myocardium'],
            'splits': {
                'train': len(X_train),
                'validation': len(X_val),
                'test': len(X_test)
            },
            'preprocessing': {
                'target_size': preprocessor.target_size,
                'normalize_method': preprocessor.normalize_method
            },
            'created': str(pd.Timestamp.now())
        }
        
        with open(os.path.join(save_path, 'metadata.json'), 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print("✅ Dados salvos com sucesso!")
    
    return {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test,
        'metadata': metadata if save_processed else None
    }

# Preparar dados
if matched_pairs:
    training_data = prepare_training_data(
        preprocessor, DATASET_PATH, matched_pairs,
        validation_split=0.2, test_split=0.1
    )
    
    if training_data:
        print("\n🎉 Dados prontos para treinamento!")
else:
    print("⚠️ Pulando preparação de dados - dataset não encontrado")
    training_data = None

⚠️ Pulando preparação de dados - dataset não encontrado


---

## 🎯 Resumo da Análise de Dados

### ✅ Tarefas Concluídas

1. **📊 Descoberta do Dataset**
   - Estrutura de diretórios analisada
   - Arquivos de imagem e labels identificados
   - Correspondência entre pares verificada

2. **🔍 Análise Estatística**
   - Dimensões e espaçamento analisados
   - Distribuição de intensidades caracterizada
   - Distribuição de classes quantificada

3. **🔬 Análise Detalhada**
   - Amostras individuais examinadas
   - Visualizações criadas
   - Propriedades espaciais verificadas

4. **🔧 Pipeline de Pré-processamento**
   - Normalização por percentis implementada
   - Redimensionamento preservando qualidade
   - Tratamento adequado de labels

5. **💾 Preparação para Treinamento**
   - Dados divididos em treino/validação/teste
   - One-hot encoding aplicado
   - Metadados salvos

### 📈 Principais Descobertas

- **Dimensões**: Variáveis, normalizadas para 128x128
- **Classes**: 3 classes com forte desbalanceamento (Background dominante)
- **Qualidade**: Dados consistentes e adequados para treinamento
- **Desafios**: Desbalanceamento de classes, variabilidade anatômica

### 🚀 Próximos Passos

1. **🔄 Data Augmentation**: Execute `02_Data_Augmentation.ipynb`
2. **🏗️ Model Architecture**: Execute `03_Model_Architecture.ipynb`
3. **📈 Loss Functions**: Execute `04_Loss_Functions_and_Metrics.ipynb`

---

**✨ Dados analisados e prontos para as próximas etapas!**